# Predict and Prevent Credit Card Payment Default: Data Cleaning Notebook

Here I use data associated with a Taiwanese Bank, which was downloaded from Google Cloud Platform, to develop a classification algorithm to predict and prevent credit card default. 

Let's start by importing the necessary libraries for data cleaning. 

In [1]:
# necessary imports

import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data cleaning

First, we load the data as a pandas dataframe and take a look at the dataset. 

In [2]:
# load the data

df = pd.read_json('/Users/laurenfinkelstein/Metis/Projects/Project 3/credit_payment_default/results-20190430-210539.json')

In [3]:
# check out the dataset

df.head()

,age,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,default_payment_next_month,education_level,id,limit_balance,marital_status,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,predicted_default_payment_next_month,sex
0,39,47174,47974,48630,50803,30789,15874,0,1,242,50000,2,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,"[{'tables': {'score': '0.85050475597381592', '...",1
1,29,48088,45980,44231,32489,26354,20221,0,1,1822,110000,2,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,"[{'tables': {'score': '0.93080854415893555', '...",2
2,36,78630,68921,46512,40335,37165,22156,0,1,5046,270000,2,0,0,0,2,0,0,10076,4018,14,2051,2000,0,"[{'tables': {'score': '0.87450653314590454', '...",1
3,45,58180,59134,61156,62377,63832,65099,0,1,7227,130000,1,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,"[{'tables': {'score': '0.90300285816192627', '...",1
4,24,42058,35340,22110,19837,19855,20151,0,1,9183,50000,2,0,0,0,0,0,0,1367,1606,692,709,721,692,"[{'tables': {'score': '0.856909453868866', 'va...",1


In [4]:
# check out the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2965 entries, 0 to 2964
Data columns (total 26 columns):
age                                     2965 non-null int64
bill_amt_1                              2965 non-null int64
bill_amt_2                              2965 non-null int64
bill_amt_3                              2965 non-null int64
bill_amt_4                              2965 non-null int64
bill_amt_5                              2965 non-null int64
bill_amt_6                              2965 non-null int64
default_payment_next_month              2965 non-null int64
education_level                         2965 non-null int64
id                                      2965 non-null int64
limit_balance                           2965 non-null int64
marital_status                          2965 non-null int64
pay_0                                   2965 non-null int64
pay_2                                   2965 non-null int64
pay_3                                   2965 non-null

Let's check if there are any duplicate entries per unique user id. 

In [5]:
# checking for duplicate entries per unique user id

check_dup = df.groupby('id').count()
check_dup[check_dup.values > 1]

,age,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,default_payment_next_month,education_level,limit_balance,marital_status,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,predicted_default_payment_next_month,sex
id,,,,,,,,,,,,,,,,,,,,,,,,,


Let's filter the dataframe to include only the columns we will use for modeling. 

In [6]:
# remove unecessary columns and reorder necessary columns

new_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 
               'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6', 
               'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 
               'limit_balance']

model_data = df[new_columns]

In [7]:
model_data.head()

,default_payment_next_month,age,sex,marital_status,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance
0,0,39,1,2,1,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000
1,0,29,2,2,1,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000
2,0,36,1,2,1,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000
3,0,45,1,1,1,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000
4,0,24,1,2,1,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000


The labels associated with each feature are described in Google Cloud Platform. However, in the dataframe we see other labels in addition the those described in GCP for some features (for example, pay_# have labels 0 and -1). Let's look at just a few of these columns to try to understand what they mean. 

In [8]:
# figuring out what the different variables mean
# pay_0 etc. have additional labels other than labels described in GCP (e.g., 0, -1), but leave as is?

temp_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1',
               'pay_0', 
               'pay_amt_1',
               'limit_balance']

df_temp = model_data[temp_columns]
df_temp.head()
# df_temp.pay_0.value_counts()

,default_payment_next_month,age,sex,marital_status,education_level,bill_amt_1,pay_0,pay_amt_1,limit_balance
0,0,39,1,2,1,47174,0,1800,50000
1,0,29,2,2,1,48088,0,2000,110000
2,0,36,1,2,1,78630,0,10076,270000
3,0,45,1,1,1,58180,0,2886,130000
4,0,24,1,2,1,42058,0,1367,50000


I'm going to assume that where pay_# columns have a negative value, the customer paid on time. Hence, I will change this value to 0. 

In [9]:
# where pay_columns have a value < 0, change to 0
# ASSUMPTION: if pay columns have a negative value, customer paid on time

pay_columns = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']

for column in pay_columns:
    model_data.loc[model_data[column] < 0, column] = 0

/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Because sex is a categorical variable, I will replace the label for female, which is 2, to be 0, in order to prepare this column for one-hot encoding. 

In [10]:
# sex: replace female=2 with female=0 to prepare for creating dummy variables

model_data['sex2'] = np.where(model_data['sex'] > 1, 0, model_data['sex']) # female = 0, male = 1

/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# checking the count of males vs. females in the dataset

model_data['sex2'].value_counts()

0    1802
1    1163
Name: sex2, dtype: int64

In [12]:
# drop the old sex column and rename the sex2 column

model_data.drop(columns=['sex'], axis=1, inplace=True)
model_data.rename(columns={'sex2': 'sex'}, inplace=True)

/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [13]:
model_data.head()

,default_payment_next_month,age,marital_status,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance,sex
0,0,39,2,1,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000,1
1,0,29,2,1,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000,0
2,0,36,2,1,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000,1
3,0,45,1,1,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000,1
4,0,24,2,1,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000,1


Similar to sex, marital status is a categorical variables, so I will replace the number representation for an individual's marital status with categorical labels to prepare for dummy encoding. 

In [14]:
# replace marital_status number representations with categorical labels to prepare for dummy encoding

model_data['marital_status'] = model_data['marital_status'].map({1: 'married', 2: 'single', 3: 'others'})
model_data.marital_status.value_counts()


/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


single     1594
married    1332
others       35
Name: marital_status, dtype: int64

Education level can be represented using an ordinal variable, so here I relabel labels ordinally, with the "others" and two "unknown" categories having a label of 0. 

In [15]:
# replace education level numerical label with string to prepare for dummy variables
# Education Level (original labels: 1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)

model_data['education_level'] = model_data['education_level'].map({1: 3, 2: 2, 3: 1, 4: 0, 5: 0, 6: 0})
model_data.education_level.value_counts()


/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


2.0    1400
3.0    1039
1.0     483
0.0      41
Name: education_level, dtype: int64

Here we one-hot encode the marital status feature using the pandas get_dummies method. 

In [16]:
dummy = pd.get_dummies(model_data['marital_status'], drop_first = True)
model_data = pd.concat([model_data,dummy], axis=1).drop(columns='marital_status')

We should now remove the few rows with null values. 

In [17]:
model_data.dropna(axis=0, inplace = True)

The data has now been cleaned and preprocessed for modeling, so let's pickle it. 

In [18]:
with open ('cleaned_cc_default_data', 'wb') as f:
    pickle.dump(model_data, f)